# 최적의 선수 선택
1. 급여를 1억 1200만달러 넘지 않는 조건
2. 클러스터별 필요 인원을 설정한 후 선수 선택
3. 선택한 선수들의 평균 평점이 가장 높은 경우의 수를 탐색

In [2]:
import pandas as pd
# 경로 설정
df_final=pd.read_csv('final_salary_cluster_injury2.csv')
df_final

,PLAYER,SEASON,TEAM,Cluster,MIN,전체적인 평점,Salary
0,Joel Embiid,2022-23,PHI,2,34.6,1.162431,33616770.0
1,Damian Lillard,2022-23,POR,2,36.3,1.404950,42492492.0
2,Shai Gilgeous-Alexander,2022-23,OKC,3,35.5,0.199302,30913750.0
3,Giannis Antetokounmpo,2022-23,MIL,3,32.1,1.059068,42492492.0
4,Jayson Tatum,2022-23,BOS,3,36.9,1.036169,30351780.0
...,...,...,...,...,...,...,...
331,Juan Toscano-Anderson,2022-23,UTA,2,13.5,-0.890270,2133278.0
332,Juancho Hernangomez,2022-23,TOR,2,14.6,-0.765831,2298385.0
333,Frank Ntilikina,2022-23,DAL,3,12.9,-1.483787,2036318.0
334,Ish Smith,2022-23,DEN,3,9.3,-0.930990,4725000.0


In [3]:
df_final.sort_values('전체적인 평점')

,PLAYER,SEASON,TEAM,Cluster,MIN,전체적인 평점,Salary
76,De'Andre Hunter,2022-23,ATL,0,31.7,-2.129888,9835881.0
50,RJ Barrett,2022-23,NYK,2,33.9,-2.063016,10900634.0
233,Romeo Langford,2022-23,SAS,0,19.6,-2.021173,5634257.0
321,Amir Coffey,2022-23,LAC,3,12.5,-1.965170,3395062.0
66,Bennedict Mathurin,2022-23,IND,5,28.5,-1.837389,6586800.0
...,...,...,...,...,...,...,...
94,Chris Paul,2022-23,PHX,1,32.0,1.996679,28400000.0
5,Stephen Curry,2022-23,GSW,3,34.7,2.005020,48070014.0
232,Ben Simmons,2022-23,BKN,4,26.3,2.093467,35448672.0
36,James Harden,2022-23,PHI,3,36.8,2.292079,33000000.0


In [4]:
import pandas as pd

# df_final을 사용한다고 가정 (선수 데이터 포함)
df = df_final.copy()

# 총 급여 한도 설정
salary_cap = 134000000

# 각 클러스터별로 필요한 인원 수 설정
cluster_requirements = {
    0: 1,  # 클러스터 0: 1명
    1: 3,  # 클러스터 1: 3명
    2: 1,  # 클러스터 2: 1명
    3: 2,  # 클러스터 3: 2명
    4: 1,  # 클러스터 4: 1명
    5: 1   # 클러스터 5: 1명
}

selected_players = []
for cluster, num_players in cluster_requirements.items():
    top_players = df[df['Cluster'] == cluster].nlargest(num_players, '전체적인 평점')
    selected_players.extend(top_players[['PLAYER', 'Salary', '전체적인 평점', 'Cluster']].to_dict('records'))

excluded_players = [p['PLAYER'] for p in selected_players]



In [5]:
selected_players

[{'PLAYER': 'Mike Conley',
  'Salary': 22680000.0,
  '전체적인 평점': 1.292903936516335,
  'Cluster': 0},
 {'PLAYER': 'Chris Paul',
  'Salary': 28400000.0,
  '전체적인 평점': 1.99667907907194,
  'Cluster': 1},
 {'PLAYER': 'Mark Williams',
  'Salary': 3722040.0,
  '전체적인 평점': 1.277074372355733,
  'Cluster': 1},
 {'PLAYER': 'Draymond Green',
  'Salary': 25806468.0,
  '전체적인 평점': 1.0665357595094906,
  'Cluster': 1},
 {'PLAYER': 'Al Horford',
  'Salary': 26500000.0,
  '전체적인 평점': 1.729690776238059,
  'Cluster': 2},
 {'PLAYER': 'Tyrese Haliburton',
  'Salary': 4215120.0,
  '전체적인 평점': 4.234981947169625,
  'Cluster': 3},
 {'PLAYER': 'James Harden',
  'Salary': 33000000.0,
  '전체적인 평점': 2.2920790910949864,
  'Cluster': 3},
 {'PLAYER': 'Ben Simmons',
  'Salary': 35448672.0,
  '전체적인 평점': 2.093467109073952,
  'Cluster': 4},
 {'PLAYER': 'Steven Adams',
  'Salary': 17926829.0,
  '전체적인 평점': 1.7630647754840922,
  'Cluster': 5}]

In [6]:
# 3. 팀의 총 급여 계산
total_salary = sum(player['Salary'] for player in selected_players)
total_salary

197699129.0

In [7]:
# 2. 가장 비싼 선수를 대체하는 함수
def replace_most_expensive_player(team, df, excluded_players):
    # 2-1. 가장 비싼 선수 찾기
    most_expensive_player = max(team, key=lambda x: x['Salary'])  # Salary 기준
    cluster = most_expensive_player['Cluster']  # 해당 선수의 클러스터

    # 2-2. 해당 클러스터에서 남은 후보들 중에서 다음으로 평점이 높은 선수 선택
    # excluded_players에 포함되지 않은 선수들만 후보에 추가
    candidates = df[(df['Cluster'] == cluster) &
                    (~df['PLAYER'].isin([p['PLAYER'] for p in team])) &
                    (~df['PLAYER'].isin(excluded_players))]
    if candidates.empty:
        return team  # 대체할 선수가 없는 경우 종료

    # 가장 높은 평점을 가진 선수 선택
    replacement = candidates.nlargest(1, '전체적인 평점').iloc[0].to_dict()

    # 2-3. 팀에서 가장 비싼 선수를 대체
    new_team = [p for p in team if p != most_expensive_player]
    new_team.append(replacement)

    return new_team, replacement['PLAYER']  # 교체된 선수 이름 반환



In [8]:
salary_cap = 134000000


In [9]:
while total_salary > salary_cap:
    selected_players, replaced_player = replace_most_expensive_player(selected_players, df, excluded_players)
    excluded_players.append(replaced_player)  # 제외 선수 목록에 추가
    total_salary = sum(player['Salary'] for player in selected_players)

In [10]:
# 5. 최종 팀의 총 평점과 평균 평점 계산
total_rate = sum(player['전체적인 평점'] for player in selected_players)
rate_avg = total_rate / len(selected_players)

# 6. 최종 팀 데이터프레임으로 변환 및 출력
final_team_df = pd.DataFrame(selected_players)
print("선택된 팀:")
print(final_team_df)
print(f"평균 평점: {rate_avg}, 총 급여: {total_salary}")


선택된 팀:
              PLAYER      Salary   전체적인 평점  Cluster   SEASON TEAM   MIN
0        Mike Conley  22680000.0  1.292904        0      NaN  NaN   NaN
1      Mark Williams   3722040.0  1.277074        1      NaN  NaN   NaN
2  Tyrese Haliburton   4215120.0  4.234982        3      NaN  NaN   NaN
3       Steven Adams  17926829.0  1.763065        5      NaN  NaN   NaN
4        Buddy Hield  21177750.0  0.910949        4  2022-23  IND  31.0
5   Domantas Sabonis  21100000.0  1.762908        3  2022-23  SAC  34.6
6      Jarrett Allen  20000000.0  0.551319        1  2022-23  CLE  32.6
7       Desmond Bane   2130240.0  1.221149        2  2022-23  MEM  31.7
8     Xavier Tillman   1782621.0  0.287542        1  2022-23  MEM  19.3
평균 평점: 1.477988025375823, 총 급여: 114734600.0


In [13]:
final_team_df.PLAYER.to_list()

['Mike Conley',
 'Mark Williams',
 'Tyrese Haliburton',
 'Steven Adams',
 'Buddy Hield',
 'Domantas Sabonis',
 'Jarrett Allen',
 'Desmond Bane',
 'Xavier Tillman']